In [51]:
import pandas as pd
import re
import numpy as np
import hashlib

In [52]:
ori_df = pd.read_csv('inputs/applications_dataset_2.csv')
df = ori_df.copy()

In [53]:
df.head()

,name,email,date_of_birth,mobile_no
0,Tony Shepherd,Tony_Shepherd@petersen.com,07/03/2016,711447
1,Sherry Gonzalez,Sherry_Gonzalez@caldwell.biz,14-03-1973,66744895
2,Ashlee Austin,Ashlee_Austin@melendez.com,12/09/1992,6454197
3,David Brown,David_Brown@jackson-smith.biz,2001-09-22,69082983
4,Marc Meyer,Marc_Meyer@chavez.com,1996/05/17,9727376


In [54]:
df.shape

(3000, 4)

In [55]:
# format mobile_no: check for only digits, length == 8
df['mobile_no'] = df['mobile_no'].apply(lambda x: ''.join(re.findall(r'\d+', x)))
df['still_successful'] = df['mobile_no'].apply(lambda x: True if len(x) == 8 else False)

In [56]:
# keep unsuccessful into a list of indexes
unsuccessful_idx = df[~df['still_successful']].index
unsuccessful_idx

Index([   0,    2,    4,    7,   11,   12,   13,   15,   16,   17,
       ...
       2988, 2989, 2990, 2993, 2994, 2995, 2996, 2997, 2998, 2999],
      dtype='int64', length=2255)

In [57]:
# df retains still successful apps
df = df[df['still_successful']].drop(columns='still_successful')
df

,name,email,date_of_birth,mobile_no
1,Sherry Gonzalez,Sherry_Gonzalez@caldwell.biz,14-03-1973,66744895
3,David Brown,David_Brown@jackson-smith.biz,2001-09-22,69082983
5,Jennifer Vega,Jennifer_Vega@jenkins.net,03/07/1971,66551251
6,Denise Glass,Denise_Glass@calderon.org,1982-05-31,77863943
8,Billy Knight,Billy_Knight@thomas.biz,01/27/1955,49833743
...,...,...,...,...
2981,Marcus Sims,Marcus_Sims@adams.com,2008-05-22,45883671
2984,Angela Nichols,Angela_Nichols@manning-krueger.com,03/15/2014,44151975
2986,Nicole Shea,Nicole_Shea@mcclure.com,1956-01-02,28795599
2991,Charles Clarke,Charles_Clarke@decker.com,2017/08/14,75379215


In [58]:
# format email: check for [alphanumerical]@[emailprovider].[com/net]
df[['first_email_part', 'second_email_part']] = df['email'].str.split('@', expand = True)
df[['emailprovider', 'com/net']] = df['second_email_part'].str.split('.', expand = True)
df['still_successful'] = df['com/net'].isin(['com', 'net'])

In [59]:
working_cols = ['first_email_part', 'second_email_part', 'emailprovider', 'com/net', 'still_successful']

In [60]:
# keep unsuccessful into a list of indexes
unsuccessful_idx = unsuccessful_idx.append(df[~df['still_successful']].index)
unsuccessful_idx

Index([   0,    2,    4,    7,   11,   12,   13,   15,   16,   17,
       ...
       2859, 2863, 2868, 2932, 2939, 2952, 2965, 2973, 2978, 2992],
      dtype='int64', length=2476)

In [61]:
# df retains still successful apps
df = df[df['still_successful']].drop(columns=working_cols)
df

,name,email,date_of_birth,mobile_no
5,Jennifer Vega,Jennifer_Vega@jenkins.net,03/07/1971,66551251
9,Shane Davis,Shane_Davis@smith.net,2006/06/25,42361783
10,Laura Harding,Laura_Harding@jackson.com,2017-12-18,32048503
21,Aaron Wilson,Aaron_Wilson@burke-benton.net,17-05-1987,75053391
29,Samuel Lee,Samuel_Lee@gardner-rodriguez.net,1974/09/02,57636831
...,...,...,...,...
2974,Kathleen Rodriguez,Kathleen_Rodriguez@frost-james.com,10/05/1967,65397891
2981,Marcus Sims,Marcus_Sims@adams.com,2008-05-22,45883671
2984,Angela Nichols,Angela_Nichols@manning-krueger.com,03/15/2014,44151975
2986,Nicole Shea,Nicole_Shea@mcclure.com,1956-01-02,28795599


In [62]:
# format name: trim whitespaces, check for first name and last name separated by a space
df['name'] = df['name'].str.strip()

In [63]:
def clean_name(name):
  split_name = name.split(' ')
  if len(split_name) == 2:
    first_name, last_name = split_name
  elif len(split_name) > 2:
    # check for salutations
    non_saluation_words = []
    for word in split_name:
      # word has more than 2 capital letters or contains .or is a known salutation
      if not (len(re.findall(r'[A-Z]', word)) >= 2 or '.' in word or word in ['Mr', 'Miss', 'Madam', 'Mdm', 'Jr']):
        non_saluation_words.append(word)
    if len(non_saluation_words) == 2:
      first_name, last_name = non_saluation_words
    else:
      first_name, last_name = None, None
  else:
    first_name, last_name = None, None

  return first_name, last_name

In [64]:
df[['first_name', 'last_name']] = df['name'].apply(clean_name).tolist()

In [65]:
# keep unsuccessful into a list of indexes
unsuccessful_idx = unsuccessful_idx.append(df[df['first_name'].isna() | df['last_name'].isna()].index)
unsuccessful_idx

Index([   0,    2,    4,    7,   11,   12,   13,   15,   16,   17,
       ...
       2859, 2863, 2868, 2932, 2939, 2952, 2965, 2973, 2978, 2992],
      dtype='int64', length=2476)

In [66]:
# df retains still successful apps
df = df[~df['first_name'].isna() & ~df['last_name'].isna()]
df

,name,email,date_of_birth,mobile_no,first_name,last_name
5,Jennifer Vega,Jennifer_Vega@jenkins.net,03/07/1971,66551251,Jennifer,Vega
9,Shane Davis,Shane_Davis@smith.net,2006/06/25,42361783,Shane,Davis
10,Laura Harding,Laura_Harding@jackson.com,2017-12-18,32048503,Laura,Harding
21,Aaron Wilson,Aaron_Wilson@burke-benton.net,17-05-1987,75053391,Aaron,Wilson
29,Samuel Lee,Samuel_Lee@gardner-rodriguez.net,1974/09/02,57636831,Samuel,Lee
...,...,...,...,...,...,...
2974,Kathleen Rodriguez,Kathleen_Rodriguez@frost-james.com,10/05/1967,65397891,Kathleen,Rodriguez
2981,Marcus Sims,Marcus_Sims@adams.com,2008-05-22,45883671,Marcus,Sims
2984,Angela Nichols,Angela_Nichols@manning-krueger.com,03/15/2014,44151975,Angela,Nichols
2986,Nicole Shea,Nicole_Shea@mcclure.com,1956-01-02,28795599,Nicole,Shea


In [67]:
# format DOB: Uses dayfirst for vague dates like 05/08/2006
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], format='mixed', dayfirst=True)

In [68]:
# calculate age and check if above 18
age_in_days = pd.to_datetime('01/01/2022', format='%d/%m/%Y', dayfirst=True) - df['date_of_birth']
age_in_years = np.floor(age_in_days / np.timedelta64(1, "Y"))
df['above_18'] = age_in_years.apply(lambda x: True if x >= 18 else False)

In [69]:
# convert DOB to string YYYYMMDD
df['date_of_birth'] = df['date_of_birth'].dt.strftime('%Y%m%d')

In [70]:
# keep unsuccessful into a list of indexes
unsuccessful_idx = unsuccessful_idx.append(df[~df['above_18']].index)
unsuccessful_idx

Index([   0,    2,    4,    7,   11,   12,   13,   15,   16,   17,
       ...
       2725, 2774, 2813, 2846, 2853, 2903, 2929, 2981, 2984, 2991],
      dtype='int64', length=2588)

In [71]:
# df retains still successful apps
df = df[df['above_18']].reset_index(drop=True)
df

,name,email,date_of_birth,mobile_no,first_name,last_name,above_18
0,Jennifer Vega,Jennifer_Vega@jenkins.net,19710703,66551251,Jennifer,Vega,True
1,Aaron Wilson,Aaron_Wilson@burke-benton.net,19870517,75053391,Aaron,Wilson,True
2,Samuel Lee,Samuel_Lee@gardner-rodriguez.net,19740902,57636831,Samuel,Lee,True
3,Jennifer Weaver,Jennifer_Weaver@patterson.com,19781015,74152151,Jennifer,Weaver,True
4,Mr. Timothy Ortega,Timothy_Ortega@ramos.com,19980401,59312963,Timothy,Ortega,True
...,...,...,...,...,...,...,...
407,Jeffrey Murray,Jeffrey_Murray@jones.net,19821221,27473887,Jeffrey,Murray,True
408,Carla Jackson,Carla_Jackson@morris.com,19940715,44275151,Carla,Jackson,True
409,Kristin Wagner,Kristin_Wagner@clayton.com,19970129,66045343,Kristin,Wagner,True
410,Kathleen Rodriguez,Kathleen_Rodriguez@frost-james.com,19670510,65397891,Kathleen,Rodriguez,True


In [72]:
# Validity checks are complete, generate Membership IDs
# Step 1: Hash birthday and truncate to first 5 digits
hashed_trunc_bdae = df['date_of_birth'].apply(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest()[:5])
# Step 2: Last name + _ + hash_trunc_bdae
df['membership_id'] = df['last_name'] + '_' + hashed_trunc_bdae
df

,name,email,date_of_birth,mobile_no,first_name,last_name,above_18,membership_id
0,Jennifer Vega,Jennifer_Vega@jenkins.net,19710703,66551251,Jennifer,Vega,True,Vega_fc7cb
1,Aaron Wilson,Aaron_Wilson@burke-benton.net,19870517,75053391,Aaron,Wilson,True,Wilson_6845c
2,Samuel Lee,Samuel_Lee@gardner-rodriguez.net,19740902,57636831,Samuel,Lee,True,Lee_d3d5c
3,Jennifer Weaver,Jennifer_Weaver@patterson.com,19781015,74152151,Jennifer,Weaver,True,Weaver_feb81
4,Mr. Timothy Ortega,Timothy_Ortega@ramos.com,19980401,59312963,Timothy,Ortega,True,Ortega_dd4da
...,...,...,...,...,...,...,...,...
407,Jeffrey Murray,Jeffrey_Murray@jones.net,19821221,27473887,Jeffrey,Murray,True,Murray_8d3f7
408,Carla Jackson,Carla_Jackson@morris.com,19940715,44275151,Carla,Jackson,True,Jackson_5ae26
409,Kristin Wagner,Kristin_Wagner@clayton.com,19970129,66045343,Kristin,Wagner,True,Wagner_83f4e
410,Kathleen Rodriguez,Kathleen_Rodriguez@frost-james.com,19670510,65397891,Kathleen,Rodriguez,True,Rodriguez_2bb86


In [73]:
unsuccessful_df = ori_df.loc[unsuccessful_idx]
unsuccessful_df

,name,email,date_of_birth,mobile_no
0,Tony Shepherd,Tony_Shepherd@petersen.com,07/03/2016,711447
2,Ashlee Austin,Ashlee_Austin@melendez.com,12/09/1992,6454197
4,Marc Meyer,Marc_Meyer@chavez.com,1996/05/17,9727376
7,Emily Christian,Emily_Christian@hardy-ball.biz,1993/11/27,83586
11,Zachary Payne,Zachary_Payne@espinoza.info,17-02-2018,7802719
...,...,...,...,...
2903,Alex Baker,Alex_Baker@kelly.com,2012-04-23,34292983
2929,Kaylee Berry,Kaylee_Berry@walsh-mckenzie.com,2016-12-24,74360527
2981,Marcus Sims,Marcus_Sims@adams.com,2008-05-22,45883671
2984,Angela Nichols,Angela_Nichols@manning-krueger.com,03/15/2014,4415 1975


In [74]:
# outputs
unsuccessful_df.to_csv('unsuccessful/unsuccessful_applicants.csv', index=False)
df.to_csv('successful/successful_applicants.csv', index=False)